In [2]:
import json

In [7]:
# Load the datasets
with open('../datasets/ita/original_ocr.json', 'r') as file:
    noisy = json.load(file)

with open('../datasets/ita/cleaned.json', 'r') as file:
    cleaned = json.load(file)

Now we tranform the keys in integer values and we sort them. This beacuse we are then going to use the first 7 chapters for evaluation and the remaining (29) for training.

In [8]:
keys = noisy.keys()
integer_keys = [int(key) for key in keys]
integer_keys.sort()

### Data preparation
The training dataset will follow the `sharegpt` format, which has the following structure:
```
[
  {
    "conversations": [
      {
        "from": "human",
        "value": "user instruction"
      },
      {
        "from": "gpt",
        "value": "model response"
      }
    ],
    "system": "system prompt (optional)",
    "tools": "tool description (optional)"
  }
]
```

In [ ]:
def prepare_datatset(keys, system_prompt, model, noisy_dataset, cleaned_dataset):
    dataset = []

    for key in keys:
        key = str(key)
        d = {"conversations": [
        {
            "from": "human",
            "value": noisy_dataset[key]
        },
        {
            "from": model,
            "value": cleaned_dataset[key]
        }
        ],
        "system": system_prompt
        }

        dataset.append(d)
    
    return dataset


In [9]:
system_prompt = 'Transform the noisy OCR text given by the user in clean text'
model = "Minerva-350"

In [ ]:
test_dataset = prepare_datatset(integer_keys[:7], system_prompt, model, 
                                noisy_dataset=noisy, cleaned_dataset=cleaned)

training_dataset = prepare_datatset(integer_keys[7:], system_prompt, model,
                                    noisy_dataset=noisy, cleaned_dataset=cleaned)

In [36]:
## Convert the dictionaries in a json file and save them
with open("../datasets/sharegpt-format-datasets/test_dataset.json", "w") as outfile:
    json.dump(test_dataset, outfile)

with open("../datasets/sharegpt-format-datasets/training_dataset.json", "w") as outfile:
    json.dump(training_dataset, outfile)

In [31]:
sample_1 = training_dataset[0]
print(sample_1['conversations'][0]['value'])

Vili. 
Geppetto  rifa  i  pi  eoli  a  Pinocchio,  e  vende  la  propria  casacca 
per  comprargli  l'Abbecedario. 
Il  burattino,  appena  che  si  fu  levata  la  fame, 
cominciò  subito  a  bofoncbiare  e  a  piangere,  per- 
chè voleva  un  paio  di  piedi  nuovi. 

Lo  lasciò  piangere  e  disperarsi  per  una  mezza  giornata. 
Ma  Geppetto,  per  punirlo  della  monelleria  fatta, 
lo  lasciò  piangere  e  disperarsi  per  una  mezza 
giornata;  poi  gli  disse: 

w. 
—  E  percliè  dovrei  rifarti  i  i^iedi  ?  Forse  per 
vederti  scappar  di  nuovo  da  casa  tua! 
—  Vi  prometto  —  disse  il  burattino  singhioz- 
zando —  elle  da  oggi  in  poi  sarò  buono.... 
—  Tutti  i  ragazzi  —  replicò  Geppetto  —  quando 
vogliono  ottenere  qualcosa,  dicono  così. 
—  Vi  prometto  che  anderò  a  scuola,  studierò 
e  mi  farò  onore.... 
—  Tutti  i  ragazzi,  quando  vogliono  ottenere 
qualcosa,  ripetono  la  medesima  storia. 
—  Ma  io  non  sono  come  gli  altri  r

In [32]:
print(print(sample_1['conversations'][1]['value']))

VIII.
Geppetto rifà i piedi a Pinocchio e vende la propria casacca per comprargli l’Abbecedario.

Il burattino, appena che si fu levata la fame, cominciò subito a bofonchiare e a piangere, perchè voleva un paio di piedi nuovi.
Lo lasciò piangere e disperarsi per una mezza giornata.

Ma Geppetto, per punirlo della monelleria fatta lo lasciò piangere e disperarsi per una mezza giornata; poi gli disse:
— E perchè dovrei rifarti i piedi? Forse per vederti scappar di nuovo da casa tua?
— Vi prometto, — disse il burattino singhiozzando, — che da oggi in poi sarò buono…
— Tutti i ragazzi — replicò Geppetto, — quando vogliono ottenere qualcosa, dicono così.
— Vi prometto che anderò a scuola, studierò e mi farò onore…
— Tutti i ragazzi, quando vogliono ottenere qualcosa, ripetono la medesima storia.
— Ma io non sono come gli altri ragazzi! Io sono più buono di tutti e dico sempre la verità. Vi prometto, babbo, che imparerò un’arte, e che sarò la consolazione e il bastone della vostra vecchiaia.

In the next step we need to prepare the `dataset_info.json` file. It has the following structure:
```
"dataset_name": {
    "file_name": "dataset_name.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "conversations",
      "system": "system",
      "tools": "tools"
    },
    "tags": {
      "role_tag": "from",
      "content_tag": "value",
      "user_tag": "user",
      "assistant_tag": "assistant"
    }
  }
```

In [10]:
# test dataset
test_info = {"dataset_name": {
        "file_name": "test_dataset.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "conversations",
            "system": "system",
        },
        "tags": {
            "role_tag": "from",
            "content_tag": "value",
            "user_tag": "human",
            "assistant_tag": model
        }
    }
}

training_info = {"dataset_name": {
        "file_name": "training_dataset.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "conversations",
            "system": "system",
        },
        "tags": {
            "role_tag": "from",
            "content_tag": "value",
            "user_tag": "user",
            "assistant_tag": model
        }
    }
}

In [11]:
## Convert the dictionaries in a json file and save them
with open("../datasets/sharegpt-format-datasets/test_info.json", "w") as outfile:
    json.dump(test_info, outfile)

with open("../datasets/sharegpt-format-datasets/training_info.json", "w") as outfile:
    json.dump(training_info, outfile)